In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import jax.numpy as jnp

import sys
import os
import importlib
import pickle
import itertools

def append_path(path):
    if path not in sys.path:
        sys.path.append(path)

def reload_module(module_name, class_name):
    module = importlib.import_module(module_name)
    importlib.reload(module)
    return getattr(module, class_name)
        
append_path(os.path.abspath(os.path.join('..', '00_utils')))
append_path(os.path.abspath(os.path.join('..', '00_utils_training')))
append_path(os.path.abspath(os.path.join('..', '00_models')))

import logging
logging.basicConfig(level=logging.ERROR, filename='error_log.txt')

import optimize_pyomo_rl

* [X] Regularization Strength with Fixed Tolerance
* [X] 'tol' with no regularization
* [X] 'tol - 'reg' cross-validation

In [26]:
ExperimentRunner = reload_module('optimize_pyomo_rl', 'ExperimentRunner')

tol = 1e-6
start_date = '2015-01-15'
extra_input = {}
extra_input['params_data'] = {'file_path': '../00_data/df_train.csv', 'start_date': start_date, 
                'n_points': 300, 'split': 200, 'n_days': 1, 'm': 1, 
                'prev_hour': False, 'prev_week': True, 'prev_year': True, 
                'spacing': 'gauss_radau',
                'encoding': {'settlement_date': 't', 'temperature': 'var1', 'hour': 'var2', 'nd': 'y'},}

extra_input['params_sequence'] = {'sequence_len': 1, 'frequency': 35}
extra_input['params_model'] = {'layer_sizes': [7, 32, 1], 'penalty': 1e-5, 'w_init_method': 'xavier'}
extra_input['params_solver'] = { 
                        "tol":tol, 
                        "dual_inf_tol": 0.1, 
                        "compl_inf_tol": tol,
                        "constr_viol_tol": 1e-8, 
                        'warm_start_init_point': 'yes',
                        "halt_on_ampl_error" : 'yes',
                        "print_level": 5, "max_iter": 3000,
                        "bound_relax_factor": 1e-8
                        }

extra_input['plot_collocation'] = True
extra_input['plot_odeint'] = True

runner = ExperimentRunner(start_date, 'default', extra_input)
runner.run()

Generating default parameters for ode solver
Running iteration 1 with parameters: 1
{'layer_sizes': [7, 32, 1], 'penalty': 1e-05, 'w_init_method': 'xavier'}
Spacing type: gauss_radau
w init method: xavier
current_16_08
y
Ipopt 3.14.16: tol=1e-06
dual_inf_tol=0.1
compl_inf_tol=1e-06
constr_viol_tol=1e-08
warm_start_init_point=yes
halt_on_ampl_error=yes
print_level=5
max_iter=3000
bound_relax_factor=1e-08


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.16, running with linear solver MUMPS 5.7.1.

Number of nonzeros in equality constraint Jacobian...:    97312
Number of nonzeros in inequality constraint Jacobian.:        0
Numbe

In [27]:
runner.results_avg

{1: {'times_elapsed': 7.092353105545044,
  'mse_odeint': 0.008492861742993279,
  'mse_coll_ode': 0.009278817948574486,
  'mse_odeint_test': 0.15129431465648754,
  'mse_coll_ode_test': 0.1550189248474055}}

In [15]:
runner.results_full

{(1,
  '2015-01-15'): {'result': {'solver_time': 7.113605976104736,
   'termination_condition': <TerminationCondition.optimal: 'optimal'>,
   'message': 'Ipopt 3.14.16\\x3a Optimal Solution Found'}, 'times_elapsed': 7.113605976104736, 'mse_odeint': 0.051888761825143265, 'mse_coll_ode': 0.0028157900853981103, 'mse_odeint_test': 0.828515628869813, 'mse_coll_ode_test': 0.7066587274568076}}

In [85]:
runner.results_avg

{1: {'times_elapsed': 7.046317052841187,
  'mse_odeint': 372.62027318280826,
  'mse_coll_ode': 0.05570399430442987,
  'mse_odeint_test': 128.96098064574062,
  'mse_coll_ode_test': 0.8033615429788682}}

----

In [20]:
ExperimentRunner = reload_module('optimize_pyomo_rl', 'ExperimentRunner')

tol = 1e-6
start_date = '2015-01-15'
extra_input = {}
extra_input['params_data'] = {'file_path': '../00_data/df_train.csv', 'start_date': start_date, 
                'n_points': 300, 'split': 200, 'n_days': 1, 'm': 1, 
                'prev_hour': False, 'prev_week': True, 'prev_year': True, 
                'encoding': {'settlement_date': 't', 'temperature': 'var1', 'hour': 'var2', 'nd': 'y'},}

extra_input['params_sequence'] = {'sequence_len': 1, 'frequency': 1}
extra_input['params_model'] = {'layer_sizes': [7, 32, 1], 'penalty': 1e-7, 'w_init_method': 'xavier'}
extra_input['params_solver'] = { 
                        "tol":tol, 
                        'warm_start_init_point': 'yes',
                        "halt_on_ampl_error" : 'yes',
                        "print_level": 5, 
                        }

extra_input['plot_collocation'] = True
extra_input['plot_odeint'] = True

runner = ExperimentRunner(start_date, 'default', extra_input)
runner.run()

Generating default parameters for ode solver
Running iteration 1 with parameters: 1
{'layer_sizes': [7, 32, 1], 'penalty': 1e-07, 'w_init_method': 'xavier'}
Spacing type: chebyshev
w init method: xavier
current_16_08
y
Ipopt 3.14.16: tol=1e-06
warm_start_init_point=yes
halt_on_ampl_error=yes
print_level=5


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.16, running with linear solver MUMPS 5.7.1.

Number of nonzeros in equality constraint Jacobian...:    97280
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:    58921

Total number of variables...............

In [21]:
runner.results_avg

{1: {'times_elapsed': 7.173947811126709,
  'mse_odeint': 0.051888761825143265,
  'mse_coll_ode': 0.0028157900853981103,
  'mse_odeint_test': 0.828515628869813,
  'mse_coll_ode_test': 0.7066587274568076}}

In [27]:
runner.save_trained_weights('rl_32_weights')

Results saved to ../00_trained_wb/rl_32_weights_2024-09-02_12-09-44.pkl


In [91]:
runner.save_results('short_training_10')

Results saved to results/short_training_10_2024-09-08_13-37-35_full.pkl
Results saved to results/short_training_10_2024-09-08_13-37-35_avg.pkl
